In [9]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
tf.__version__

'1.12.0'

In [10]:
def plot_loss_history(histories, key='loss'):
    plt.figure(figsize=(8,5))

    for name, history in histories:
        val = plt.plot(history.epoch, history.history['val_'+key],
                       '--', label=name.title()+' Val')
        plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
                 label=name.title()+' Train')

    plt.xlabel('Epochs')
    plt.ylabel(key.replace('_',' ').title())
    plt.legend()

    plt.xlim([0,max(history.epoch)])

In [11]:
def plot_accuracy_history(histories, key='accuracy'):
    plt.figure(figsize=(8,5))

    for name, history in histories:
        val = plt.plot(history.epoch, history.history['val_'+key],
                       '--', label=name.title()+' Val')
        plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
                 label=name.title()+' Train')

    plt.xlabel('Epochs')
    plt.ylabel(key.replace('_',' ').title())
    plt.legend()

    plt.xlim([0,max(history.epoch)])

In [12]:
# load data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
train_img_size = 2500
for i in range(1, train_img_size+1):
    img = load_img('./CIFAR10/finaltest/test ('+str(i)+').png')
    if i == 1:
        x_final = [img_to_array(img)]
    else:
        x_final = np.append(x_final, [img_to_array(img)], axis=0)


# reshape
x_train = x_train.reshape(x_train.shape[0], 32, 32, 3)
x_test = x_test.reshape(x_test.shape[0], 32, 32, 3)
x_final = x_final.reshape(x_final.shape[0], 32, 32, 3)

# Making sure that the values are float so that we can get decimal points after division
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_final = x_final.astype('float32')

In [13]:
print('x_train shape:', x_train.shape)
print('Number of images in x_train', x_train.shape[0])
print('Number of images in x_test', x_test.shape[0])
print('Number of images in x_final', x_final.shape[0])

x_train shape: (50000, 32, 32, 3)
Number of images in x_train 50000
Number of images in x_test 10000
Number of images in x_final 2500


In [14]:
y_train

array([[6],
       [9],
       [9],
       ...,
       [9],
       [1],
       [1]], dtype=uint8)

In [30]:
# one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

# normalize -1~1
x_train = (x_train-127.5)/127.5
x_test = (x_test-127.5)/127.5
x_final = (x_final-127.5)/127.5

# validation
x_val = x_train[-1000:]
y_val = y_train[-1000:]
x_train = x_train[:-1000]
y_train = y_train[:-1000]

In [7]:
# set up image augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=15,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
    #zoom_range=0.3
)
datagen.fit(x_train)

In [33]:
lrate = 5e-4
epochs = 100
bsize = 128

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(32,32,3)),
    tf.keras.layers.BatchNormalization(axis=-1),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(axis=-1),
    tf.keras.layers.MaxPooling2D((2,2)),
#     tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(axis=-1),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(axis=-1),
    tf.keras.layers.MaxPooling2D((2,2)),
#     tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(axis=-1),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(axis=-1),
    tf.keras.layers.MaxPooling2D((2,2)),
#     tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.summary()
optimizer = tf.keras.optimizers.Adam(lr=lrate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
# baseline_history = model.fit(x_train, y_train, epochs=epochs, batch_size=bsize, validation_data=(x_val, y_val))

# train with image augmentation
augmentation_history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=bsize), steps_per_epoch = len(x_train) / bsize, epochs=epochs, validation_data=(x_val, y_val))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_7 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_8 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
batch_normalization_9 (Batch (None, 16, 16, 64)        256       
__________

In [34]:
result = model.evaluate(x_test,y_test,batch_size=128)

10000/10000 [==============================] - 1s 69us/step


In [35]:
print("evaluate result: ", result)

evaluate result:  [0.48752272262573243, 0.8714]


In [36]:
predictions = model.predict(x_test)
print('predictions shape:', predictions.shape)

predictions shape: (10000, 10)


In [37]:
import csv
predictions = model.predict(x_final)
print('predictions shape:', predictions.shape)

with open('CIFAR10_result.csv', mode='w') as result_file:
    writer = csv.writer(result_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['ID', 'Class'])
    i = 1
    for p in predictions:
        writer.writerow([i, np.argmax(p)])
        i += 1

predictions shape: (2500, 10)
